# Slice setup

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

## Slice

In [2]:
# slice = fablib.new_slice(name="P4DPDK_slice")

In [3]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab4_vnic")
# slice.delete()

In [4]:
servers = []

servers.append(slice.get_node(name="server1"))     
# servers.append(slice.get_node(name="server2"))
# servers.append(slice.get_node(name="server3"))

server1 = servers[0]
# server2 = servers[1]
# server3 = servers[2]

## Reboot

In [16]:
for server in servers:
    server.os_reboot()

## Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/SYN/figs/07_getting_interfaces.png" width="550"><br>

In [6]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()+'np0'
print(f'server1_iface: {server1_iface_name}')

node2_iface1 = server2.get_interface(network_name='net1') 
server2_iface1_name = node2_iface1.get_device_name()+'np0'
print(f'server2_iface1: {server2_iface1_name}')

node2_iface2 = server2.get_interface(network_name='net2') 
server2_iface2_name = node2_iface2.get_device_name()+'np1'
print(f'server2_iface2: {server2_iface2_name}')

node3_iface = server3.get_interface(network_name='net2') 
server3_iface_name = node3_iface.get_device_name()+'np0'
print(f'server3_iface: {server3_iface_name}')

server1_iface: enp7s0np0
server2_iface1: enp7s0np0
server2_iface2: enp8s0np1
server3_iface: enp7s0np0


In [18]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface1_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface2_name} up', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)

In [19]:
server1_iface_MAC = '00:00:00:00:00:01'
server2_iface1_MAC = '00:00:00:00:00:21'
server2_iface2_MAC = '00:00:00:00:00:22'
server3_iface_MAC = '00:00:00:00:00:03'

In [20]:
server1_server2_subnet = "192.168.10.0/24"
server1_ip = '192.168.10.1/24'
server2_1_ip = '192.168.10.2/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} {server2_1_ip}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} hw ether {server2_iface1_MAC}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [21]:
server2_server3_subnet = "192.168.20.0/24"
server2_2_ip = '192.168.20.2/24'
server3_ip = '192.168.20.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} {server2_2_ip}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_ip}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} hw ether {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [22]:
stdout, stderr = server2.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)

In [23]:
ip1 = server1_ip.split('/')[0]
ip2_1 = server2_1_ip.split('/')[0]
ip2_2 = server2_2_ip.split('/')[0]
ip3 = server3_ip.split('/')[0]
subnet1 = "192.168.10.0/24"
subnet2 = "192.168.20.0/24"

stdout, stderr = server1.execute(f'sudo arp -s {ip2_1} {server2_iface1_MAC}')
stdout, stderr = server1.execute(f'sudo ip route add {subnet2} via {ip2_1}')

stdout, stderr = server2.execute(f'sudo arp -s {ip1} {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {ip3} {server3_iface_MAC}')

stdout, stderr = server3.execute(f'sudo arp -s {ip2_2} {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ip route add {subnet1} via {ip2_2}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [7]:
stdout, stderr = server1.execute(f'sudo ibdev2netdev')
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)
stdout, stderr = server1.execute(f'sudo mst start', quiet=True)
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)

stdout, stderr = server2.execute(f'sudo ibdev2netdev')
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)
stdout, stderr = server2.execute(f'sudo mst start', quiet=True)
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)

stdout, stderr = server3.execute(f'sudo ibdev2netdev')
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)
stdout, stderr = server3.execute(f'sudo mst start', quiet=True)
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)

mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server1: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Up)
 sudo: unable to resolve host server2: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server3: Name or service not known
 

# Get SSH

In [5]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fec7:715'

# Upload and Compile

In [28]:
threads = []

for server in servers:
    threads.append(server.execute_thread(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"'))

for thread in threads:
    thread.result()

## Reflector

In [15]:
server2.upload_file('experiments/reflector/reflector.p4','reflector.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna reflector.p4 -o reflector.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [16]:
server2.upload_file('experiments/reflector/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/reflector/reflector.cli','reflector.cli')

<SFTPAttributes: [ size=556 uid=1000 gid=1000 mode=0o100664 atime=1729096560 mtime=1729096560 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/reflector.cli

In [17]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:fe79:77b8'

In [57]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:01
    set 0 dst mac 00:00:00:00:00:21
    set 0 src ip 192.168.10.1/24
    set 0 dst ip 192.168.10.2
    set 0 size 1500

In [18]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe8d:7886'

In [31]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.20.2
    set 0 size 1500

## Heavy Hitter

In [29]:
server2.upload_file('experiments/hh/hh.p4','hh.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna hh.p4 -o hh.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [8]:
server2.upload_file('experiments/hh/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/hh/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/hh/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/hh/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/hh/hh.cli','hh.cli')

<SFTPAttributes: [ size=907 uid=1000 gid=1000 mode=0o100664 atime=1729192859 mtime=1729192860 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/hh.cli

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 1.1.1.1
    range 0 dst ip min 1.1.1.1
    range 0 dst ip max 250.250.250.250
    range 0 dst ip inc 0.0.0.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500
    range 0 size inc 0
    
    enable 0 range
    page main

In [11]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.20.2
    set 0 size 1500
    set 0 rate 1

## SYN Flood

In [13]:
server2.upload_file('experiments/syn/syn.p4','syn.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna syn.p4 -o syn.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [14]:
server2.upload_file('experiments/syn/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/syn/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/syn/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/syn/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/syn/syn.cli','syn.cli')

server2.upload_file('experiments/syn/rules.txt','rules.txt')
server2.upload_file('experiments/syn/reset_SYN_packets_per_second.py','reset_SYN_packets_per_second.py')

<SFTPAttributes: [ size=1183 uid=1000 gid=1000 mode=0o100664 atime=1729194058 mtime=1729194058 ]>

In [15]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe30:337f'

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/syn.cli

in a new terminal:
    
    sudo python3 reset_SYN_packets_per_second.py

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range
    
    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500
    
    range 0 tcp flag clr all
    range 0 tcp flag set syn
    
    enable 0 range
    page main

In [11]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

In [16]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe30:337f'

    telnet 0.0.0.0 8086

    pipeline PIPELINE0 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 50 index 0
    
    pipeline PIPELINE0 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 100 index 0

## Interarrival Time

In [6]:
# stdout, stderr = server2.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
# server2.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
# server2.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
# stdout, stderr = server2.execute(f'cd dpdk && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

In [43]:
server1.upload_file('experiments/iat/test_iat.p4','test_iat.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna test_iat.p4 -o test_iat.spec')

 sudo: unable to resolve host server1: Name or service not known
 

In [12]:
server1.upload_file('experiments/iat/ethdev0.io','ethdev0.io')
# server2.upload_file('experiments/iat/ethdev1.io','ethdev1.io')
# server2.upload_file('experiments/iat/ethdev2.io','ethdev2.io')
# server2.upload_file('experiments/iat/ethdev3.io','ethdev3.io')
server1.upload_file('experiments/iat/iat.cli','iat.cli')

server1.upload_file('experiments/iat/rules.txt','rules.txt')

<SFTPAttributes: [ size=112 uid=1000 gid=1000 mode=0o100664 atime=1725381418 mtime=1729788055 ]>

In [44]:
server1.upload_file('experiments/iat/get_iat.py','get_iat.py')

<SFTPAttributes: [ size=3134 uid=1000 gid=1000 mode=0o100664 atime=1729790332 mtime=1729802412 ]>

In [31]:
server1.upload_file('experiments/iat/recv.py','recv.py')
server1.upload_file('experiments/iat/send.py','send.py')

<SFTPAttributes: [ size=4042 uid=1000 gid=1000 mode=0o100664 atime=1729794599 mtime=1729794599 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/iat.cli
    sudo python3 get_iat.py 

In [45]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

In [24]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 ping 192.168.10.2 -c 4')

 sudo: unable to resolve host server1: Name or service not known
ping: connect: Network is unreachable
 

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.10.1
    set 0 size 1500    
    set 0 rate 0.001
    

    sudo hping3 --flood 192.168.10.1
    sudo ping 192.168.20.2 -i 1

## Heavy Hitter and SYN Flood

In [12]:
server2.upload_file('experiments/hh_syn/hh_syn.p4','hh_syn.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna hh_syn.p4 -o hh_syn.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [13]:
server2.upload_file('experiments/hh_syn/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/hh_syn/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/hh_syn/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/hh_syn/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/hh_syn/hh_syn.cli','hh_syn.cli')

server2.upload_file('experiments/hh_syn/rules.txt','rules.txt')

<SFTPAttributes: [ size=112 uid=1000 gid=1000 mode=0o100664 atime=1729615818 mtime=1729615980 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/hh_syn.cli

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500

    range 0 tcp flag clr all
    range 0 tcp flag set syn
    
    enable 0 range
    page main
    

In [11]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.10.1
    set 0 size 1500
    set 0 rate 0.5

# Step 10: Delete the slice

This concludes Lab 9. Please delete your slice when you are done with your experiment.

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_HH4")
# slice.delete()